In [1]:
import datetime as dt
import panel as pn

import numpy as np
import pandas as pd

import bokeh.io

from bokeh.tile_providers import CARTODBPOSITRON, get_provider

tile_provider = get_provider(CARTODBPOSITRON)

pn.extension()

# Load in the metadata
ferm_locations = pd.read_csv(("./data/ferm.csv"))

# Split the Lat-Long column into two columns, Lat and Long
ferm_locations[['Lat', 'Long']] = ferm_locations['Lat-Long'].str.split(', ', 1, expand=True)

# Create a new dataframe only if the point has a valid Lat and Long
ferm_locations_final = ferm_locations[ferm_locations['Lat'].notna()].copy()

# Convert from a string, and put in columns called Latitude and Longitude
ferm_locations_final.loc[:,'Latitude1'] = pd.to_numeric(ferm_locations_final['Lat'],errors='coerce')
ferm_locations_final.loc[:,'Longitude1'] = pd.to_numeric(ferm_locations_final['Long'],errors='coerce')


def mercator_creator(df, lat='lat', lon='lon'):
    # from https://www.youtube.com/watch?v=BojxegBh9_4
    
    k = 6378137
    df['x'] = k * np.radians(df[lon])
    df['y'] = np.log(np.tan((90 + df[lat]) * np.pi / 360)) * k
    
    return df

ferm_locations_final = mercator_creator(ferm_locations_final, lat='Latitude1', lon='Longitude1')
ferm_locations_final.Acquisition_date = pd.to_datetime(ferm_locations_final.Acquisition_date)
ferm_locations_final.Extract_date = pd.to_datetime(ferm_locations_final.Extract_date)
ferm_locations_final.Arrival_date = pd.to_datetime(ferm_locations_final.Arrival_date)


In [2]:
def extract_sub_df(df, ferm_class, country, date_range):
    """Extract sub data frame for country and class over a date range."""
    inds = (
        (df["Class"] == ferm_class)
        & (df["Country_made"] == country)
        & (df["Acquisition_date"] >= date_range[0])
        & (df["Acquisition_date"] <= date_range[1])
    )

    return df.loc[inds, :]


date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2021, 12, 1), end=dt.datetime(2022, 8, 1),
    value=(dt.datetime(2021, 12, 1), dt.datetime(2022, 8, 1))
)



def plot_map_bokeh(ferm_locations_final, ferm_class, country, date_range=date_range_slider.value):
    """Make a map in bokeh."""

    sub_df = extract_sub_df(ferm_locations_final, ferm_class, country, date_range)

    p = bokeh.plotting.figure(
        frame_height=400,
        frame_width=700,
        x_axis_label="Latitude",
        y_axis_label="Longitude",
        tools =['wheel_zoom', 'pan', 'reset', 'tap'],
        x_axis_type = 'mercator',
        y_axis_type = 'mercator',
        x_range = [ferm_locations_final.x.min()-100000, ferm_locations_final.x.max() + 100000],
        y_range = [ferm_locations_final.y.min()-100000, ferm_locations_final.y.max() + 100000]

    )

    # Set up data source; this is what gets changed in the callback
    source = bokeh.models.ColumnDataSource(
        dict(
            x=sub_df["x"].values,
            y=sub_df["y"].values,
            date=sub_df["Acquisition_date"].values,
            Class=sub_df['Class'].values
        )
    )
    
    Class_list=ferm_locations_final.Class.unique()
    Class_len = len(Class_list)

    # Mapping of color for glyphs
    mapper = bokeh.transform.factor_cmap('Class', palette=bokeh.palettes.Category20[15], factors=Class_list)

    p.circle(source=source, x="x", y="y", color=mapper, size=7, line_alpha=0)

    p.toolbar_location = 'above'
    p.add_tile(tile_provider)
    p.add_tools(bokeh.models.BoxZoomTool(match_aspect=True))
    
    hover = bokeh.models.HoverTool()
    hover.tooltips = """
    <font face="Arial" size="3">
    <div>
        <div><strong>Ferment Class:  </strong>@Class</div>
        <div><strong>Name: </strong>@x</div>
    </div>
    </font>
    """
    p.add_tools(hover)
    
    url = "https://github.com/lianamerk/hovercal/blob/main/examples/fruit_hovercal_1.png?raw=true"
    taptool = p.select(type=bokeh.models.TapTool)
    taptool.callback = bokeh.models.OpenURL(url=url)

    return p

ferm_class = 'Dairy'
country = 'France'

p_pane = pn.pane.Bokeh(plot_map_bokeh(ferm_locations_final, ferm_class, country))

def date_interval_callback(target, event):
    date_range = event.new

    inds = (
        (ferm_locations_final["Class"] == ferm_class)
        & (ferm_locations_final["Country_made"] == country)
        & (ferm_locations_final["Acquisition_date"] >= date_range[0])
        & (ferm_locations_final["Acquisition_date"] <= date_range[1])
    )

    sub_df = ferm_locations_final.loc[inds, ["x", "y", "Acquisition_date", "Class"]]

    # print(len(sub_df))
    gr = target.object.renderers[0]
    source = gr.data_source
    source.data["x"] = sub_df["x"].values
    source.data["y"] = sub_df["y"].values
    source.data["date"] = sub_df["Acquisition_date"].values
    source.data["Class"] = sub_df["Class"].values
    
    # print(len(source.data["x"]), len(source.data["x"]), len(source.data["date"]), len(source.data["Class"]))
    
date_range_slider.link(p_pane, callbacks={'value': date_interval_callback})
map_panel = pn.Column(date_range_slider, pn.Spacer(height=15), p_pane)

In [3]:
date_range_slider.link(p_pane, callbacks={'value': date_interval_callback})

Watcher(inst=DateRangeSlider(end=datetime.datetime(2022, ..., name='Date Range Slider', start=datetime.datetime(2021, ..., value=(datetime.datetime(2021, ..., value_end=datetime.datetime(2022, ..., value_start=datetime.datetime(2021, ...), cls=<class 'panel.widgets.slider.DateRangeSlider'>, fn=<function Reactive.link.<locals>.link at 0x7f8f69c04af0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [4]:
map_panel = pn.Column(date_range_slider, pn.Spacer(height=15), p_pane)

In [5]:
map_panel

Column
    [0] DateRangeSlider(end=datetime.datetime(2022, ..., name='Date Range Slider', start=datetime.datetime(2021, ..., value=(datetime.datetime(2021, ..., value_end=datetime.datetime(2022, ..., value_start=datetime.datetime(2021, ...)
    [1] Spacer(height=15)
    [2] Bokeh(Figure)